In [1]:
print("OK")

OK


In [2]:
%pwd

'c:\\Users\\surya\\CampusX\\End-to-end-medical-chatbot\\research'

In [3]:
%cd ..

c:\Users\surya\CampusX\End-to-end-medical-chatbot


In [4]:
%pwd

'c:\\Users\\surya\\CampusX\\End-to-end-medical-chatbot'

In [5]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)
    document = loader.load()

    return document

In [7]:
extracted_data = load_pdf_file(data='Data/')

In [8]:
len(extracted_data)

759

In [9]:
def text_split(extracted_data):
    splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
    chunks = splitter.split_documents(extracted_data)

    return chunks

In [10]:
text_chunks = text_split(extracted_data)

In [11]:
print(f'Length of text chunks: {len(text_chunks)}')

Length of text chunks: 3745


In [12]:
from dotenv import load_dotenv
import os
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [13]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [14]:
len(embedding_model.embed_query('Hello'))

768

In [15]:
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [16]:
from pinecone.grpc import PineconeGRPC as Pinecone 
from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

name = 'medicalbot'

pc.create_index(
    name=name,
    dimension=768,
    metric='cosine',
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
)

In [20]:
from langchain.vectorstores import Pinecone

docsearch = Pinecone.from_documents(
    documents = text_chunks,
    index_name = name,
    embedding = embedding_model
)


In [21]:
docretrieve = docsearch.from_existing_index(
    index_name=name,
    embedding=embedding_model
)

In [22]:
retriever = docretrieve.as_retriever(search_type = 'similarity',search_kwargs = {'k':5})

In [23]:
retrived_docs = retriever.invoke("What is rosacea")

In [24]:
retrived_docs

[Document(metadata={'page': 425.0, 'source': 'Data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf'}, page_content='the irritation. Oral antihistamines are sometimes recom-\nmended to alleviate itching, and antibiotics are pre-\nscribed if the rash becomes infected. Medications taken\nby mouth to relieve symptoms of dermatitis can make\nskin red and scaly and cause hair loss.\nKEY TERMS\nAllergic reaction—An inappropriate or exaggerat-\ned genetically determined reaction to a chemical\nthat occurs only on the second or subsequent\nexposures to the offending agent, after the first\ncontact has sensitized the body.\nCorticosteriod—A group of synthetic hormones\nthat are used to prevent or reduce inflammation.\nToxic effects may result from rapid withdrawal after\nprolonged use or from continued use of large doses.\nPatch test—A skin test that is done to identify aller-\ngens. A suspected substance is applied to the skin.\nAfter 24–48 hours, if the area is red and swollen,\nthe test is posi

In [26]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model='gemini-1.5-pro',max_tokens=500,temperature=0.3)

In [30]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [31]:
system_prompt = (
    "You are an medical assistant for question-answering task"
    "Use the following pieces of retrieved context for answering the questions"
    "If you don't know the answer say I don't know "
    "Use five sentences maximum and keep the answer concise"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]

)

In [32]:
question_answer_chain = create_stuff_documents_chain(llm,prompt=prompt)

rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [35]:
response = rag_chain.invoke({"input":"What is Diabetes"})
print(response['answer'])

Diabetes mellitus is a chronic disease characterized by the body's inability to produce or respond properly to insulin.  Insulin is needed to convert glucose (sugar) into energy.  Symptoms include frequent urination, lethargy, excessive thirst, and hunger. Treatment includes dietary changes, oral medication, and sometimes insulin injections.  Diabetes can lead to serious complications like kidney failure, heart disease, stroke, and blindness.

